## CS 410 - FINAL PROJECT
##### Thomas Downs

### INTRO

Feel free to run this from the top! It doesn't do anything too demanding, and all the default values for the dataframe and dictionary generation keep N <= 10000. The only required libraries are below, and if you have worked with Python in any capacity you likely have everything except for RapidFuzz already installed. If you don't want to install RapidFuzz, just know that the portions that use a string similarity metric will fail (SIMILAR QUERY TOKEN SUBSTITUTION and DOCUMENT RANKING NAME SIMILARITY TIEBREAKERS). However, if you have other Python libraries that have string similarity metrics, or have other string similarity metrics that you'd want to use, you should be able to replace JaroWinkler.similarity in the function calls and everything will work just as well.

### IMPORTS

In [1]:
import pandas as pd
import numpy as np
from math import ceil, floor, log
from collections import defaultdict
from rapidfuzz.distance import JaroWinkler

### NAME GENERATOR

The first goal of this project is to generate a baseline collection of names to use as a corpus. Due to not wanting to use any personally identifiable information (PII) I opted to create a basic mixture model to generate names, consisting of three unigram models. I found the 200 most common male and female first names, as well as the 400 most common (actually 397 because I copied it wrong) last names, both from government data for the year 2010. Then, I used the counts to create a basic probability function for each of the names and created a function generate n full names (first name + last name), approximately n/2 of which will be masculine and n/2 of which will be feminine.
<br>
<br>
#### KEY ASSUMPTIONS 
- anglocentric names
- the odds of any first name + last name are independent
- no middle names, initials, hyphens, non-alphabetic characters, etc.

sources:  
https://www.ssa.gov/oact/babynames/decades/names2010s.html  
https://www.census.gov/topics/population/genealogy/data/2010_surnames.html

In [2]:
# load CSVs
data_types = {'name': 'string', 'count': 'int'}
df_fem_nms = pd.read_csv("female_first_200.csv", dtype=data_types)
df_masc_nms = pd.read_csv("male_first_200.csv", dtype=data_types)
df_surnames = pd.read_csv("last_names_397.csv", dtype=data_types)

In [3]:
# data cleaning
df_fem_nms.name = df_fem_nms.name.str.upper()
df_masc_nms.name = df_masc_nms.name.str.upper()

In [4]:
# calculate unigram probabilities for all names
df_fem_nms['prob'] = df_fem_nms['count'] / sum(df_fem_nms['count'])
df_masc_nms['prob'] = df_masc_nms['count'] / sum(df_masc_nms['count'])
df_surnames['prob'] = df_surnames['count'] / sum(df_surnames['count'])

In [5]:
def generate_full_names(n):
    # generate male names
    masc_firsts = np.random.choice(df_masc_nms.name, floor(n/2), p=df_masc_nms.prob)
    # generate female names
    fem_firsts = np.random.choice(df_fem_nms.name, ceil(n/2), p=df_fem_nms.prob)
    # generate last names
    surnames = np.random.choice(df_surnames.name, n, p=df_surnames.prob)
    # zip results
    list_nms = list(zip(np.append(masc_firsts, fem_firsts), surnames))
    # concat first + last and return
    return list(map(lambda x: x[0] + " " + x[1], list_nms))

In [6]:
generate_full_names(100)[:5]

['JOSEPH CERVANTES',
 'RYAN FOX',
 'STEVEN WILLIAMS',
 'SAMUEL JOHNSON',
 'EVERETT CLARK']

### NAME + ID GROUPINGS

Next, I add some duplication to the data to ensure that it works for groups (documents) with more than one name. I also add an ID to act as the primary key to group the different names into documents. The goal (and intent) of this project is to group these names with social security numbers (SSNs) as the primary key, but again to avoid any PII (or even appearance of using PII in any capacity), I opted into 1 through N numeric IDs instead. This serves the same purpose of providing a clear way to group the names into separate documents, while clearly not being real-world data.  

The duplication is pretty basic; based on the percentage of duplicate names parameter, some percentage of the names will be duplicated once. There is a chance that duplicate names are generated from the mixture model as well, but for smaller corpuses this is fairly unlikely. After that, some percentage of the IDs are also duplicated. This is done by randomizing the existing IDs and duplicating some number of them based on percentage duplicate ID. This is done for 2 rounds, although more can be specified if wanted. Note that the maximum number of times an ID can be duplicated is 2^(rounds), and I decided that having between 1 to 4 duplicate IDs should be enough for my purposes.
<br>
<br>
#### KEY ASSUMPTIONS
- names with the same ID are not more likely to be similar. This does NOT reflect real world data, where misspellings, relations, changed first/last names are all much more likely to be a different name with the same ID (remember the ID is an SSN stand-in)
- names are not duplicated based on their rate of occurance in the real world (although any naturally occuring duplicates would be)

In [7]:
# for some number of names, add a copy of it to the database
# assign each a row number
# for some number of names (iterations?), subtract 1 from id number

In [8]:
def generate_name_id_corpus(n, perc_duplicate_nm=.2, perc_duplicate_id=.1, rounds_duplicate_id=2):
    # generate names
    list_nms = generate_full_names(ceil(n * (1-perc_duplicate_nm)))  # make 1 - perc_duplicate_nm original names (still possible duplicates but rare)
    list_nms += list_nms[:floor(n * (perc_duplicate_nm))]  # append copy perc_duplicate_nms from front of list to end of list
    df = pd.DataFrame(list_nms, columns=['name'])

    # generate ids
    n_dupl = floor(n * perc_duplicate_id)  # get amount to duplicate per round
    ids = list(df.index)  # gen initial list of ids

    for _ in range(rounds_duplicate_id):
        np.random.shuffle(ids)  # shuffle existing list
        ids[:len(ids)-n_dupl-1:-1] = ids[:n_dupl]  # and make the last n_duplicate = first n_duplicate

    df['id'] = ids  # assign duplicated ids to df
    return df

### CREATE CORPUS

Actually create the corpus! A combination of generating the names, duplicating them, then generating, duplicating, and adding IDs.

In [9]:
df_corpus = generate_name_id_corpus(10000)

In [10]:
df_corpus.head()

,name,id
0,COOPER LOPEZ,8241
1,JACE WILLIAMS,6112
2,CAMERON LYONS,3948
3,LANDON LONG,1544
4,KENNETH SILVA,5886


### CREATE TEST DATA

I also have code here to generate test data, although I did not end up using it further in the process. If there was a need to do evaluation of the accuracy of document ranking or string similarity algorithms, this would serve as a way to provide names that were both form the original corpus and generated from the same model but not necessarily from the corpus to gauge the performance on both.

In [11]:
def generate_name_id_test(n, df_corpus, perc_from_corpus=.5):
    # sample from corpus for some perc, generate the rest using same function
    n_new = floor(n * (1 - perc_from_corpus))
    df_n = generate_name_id_corpus(n_new)  # new data
    df_n['source'] = "new"

    n_corpus = ceil(n * perc_from_corpus)
    df_c = df_corpus[:n_corpus].copy()  # corpus data
    df_c['source'] = "corpus"

    return pd.concat([df_n, df_c]).reset_index(drop=True)

In [12]:
df_test = generate_name_id_test(100, df_corpus)

In [13]:
df_test.head()

,name,id,source
0,NATHAN HUDSON,42,new
1,JONATHAN WEBER,12,new
2,AVERY PEREZ,46,new
3,BENJAMIN CRUZ,3,new
4,ANDREW BAKER,4,new


### ID -> TOKEN AND TOKEN -> ID CORPUS DICTS

Here I create two indices in the form of dictionaries: a list of all tokens per each ID, and a list of all IDs for each token. The list of all IDs for each token provides a simple way to get a list of all documents that should be considered based on the query text, and the list of all tokens for each ID is key for calculating IDF, as well as for having a list of all tokens that exist in the corpus.  
<br>
#### KEY ASSUMPTIONS
- this is inefficient at scale, iterating row-by-row is almost never the right way to do something like this. Larger-scale solutions would require a more distributed or database-centric approach

In [14]:
df_corpus['tokens'] = df_corpus.name.str.split()

In [15]:
id_token_dict = {}
token_id_dict = defaultdict(list)

In [16]:
for i, row in df_corpus.groupby('id')[['tokens']].sum().iterrows():
    id_token_dict[i] = list()
    for token in set(row.tokens):
        id_token_dict[i].append(token)  # dict of every token related with a specific ID
        token_id_dict[token].append(i)  # dict of every ID related with a specific token

### TF / IDF COUNTER

Separate TF / IDF Counter objects or dictionaries are not required, since the needed values can be derived from the above dictionaries.

In [17]:
# TF = 1 (since working w/ sets)
# IDF = 1 / len(token_id_dict[TOKEN])

### SIMILAR QUERY TOKEN SUBSTITUTION

The first use of string similarity metrics in the process, here I create a function to account for misspellings or unknown tokens (although Jelinek-Mercer smoothing also accounts for non-existent tokens!). First, the query is broken into separate name tokens, and each is check against the list of all tokens in the corpus. If the token does exist, it is kept as is. If the token does NOT exist, then a string similarity function is used to compare the token from the query with all the tokens in the corpus, the most similar corpus token is chosen, and the missing query token is replaced. This serves as a basic spell check, as well as accounting for any names that may have rare or unusual spellings (not uncommon for names!).  

For both this token substitution and the name similarity tiebreakers later I use the Jaro-Winkler simlarity score. Jaro-Winkler is a well regarded string matching algorithm, especially for names, although it still returns some unintuitive results (as any string similarity metric does!). The functions I wrote only expect a two-argument similarity function as an argument, rather than insisting on Jaro-Winkler, so there is room for further experimentation if wanted.  
<br>
#### KEY ASSUMPTIONS:
- that the corpus will have some token that is similar to the misspelled or rare token in the query. Since it is generated, there is even a chance that names in the source CSVs don't actually appear in the corpus
- that Jaro-Winkler is the best string similarity metric to use! In my opinion it would be the best for a real world dataset, but any similarity metric is going to have issues given that it is matching against a small corpus (at MOST ~800 unique tokens)

In [18]:
def substitute_similar_tokens(orig_query, token_id_dict, similarity_function):
    new_query = ""

    for query_token in orig_query.split():  # for each otken in query...
        token_comps = []
        new_token = ""
        if query_token not in token_id_dict:  # check if exists in corpus
            for doc_token in token_id_dict.keys():  # if not...
                token_comps.append((doc_token, similarity_function(query_token, doc_token)))  # get similarity score with all corpus tokens
            new_token = sorted(token_comps, key=lambda x: x[1], reverse=True)[0][0]  # return corpus token with closest match
            new_query += new_token
        else:
            new_query += query_token
        new_query += " "

    return new_query.strip()  # return new query, removing trailing whitespace

In [19]:
substitute_similar_tokens("JOOHN SMIITH", token_id_dict, JaroWinkler.similarity)

'JOHN SMITH'

### JELINEK-MERCER SMOOTHING

Implementation of the Jelinek-Mercer (JM) Smoothing from the lecture, assuming that each token is from a set (so TF = 1)

In [20]:
# for each word in query, need...
# count word in the document // token->id dict only returns ids w/ token
# len document // len(id_token_dict[id])
# count word in corpus // len(token_id_dict[token]

In [21]:
def jm_smoothing(len_doc, prob_token_in_corp, lmb=.2):
    return log(1 + (1-lmb) / (lmb * len_doc * prob_token_in_corp))

### DOCUMENT RANKING

Use the JM Smoothing algorithm to provide a score for each document based on the tokens in the query. For this search, a document is considered as all the names that share the same ID (SSN in a real world application), and the query is the provided name to be searched. A real world implementation could also apply similarity matching to the IDs as well, for example only including IDs in the document aggregation that have a Hamming distance of 2 or less from a query ID or IDs provided along with the query name.  
  
For this implementation, I take the top 10 results based on the JM Smoothing ranking and note the relevant scores. This is used in the next step for checking the string similarity between the query and the document names as a tie breaker and for further ranking of relevant results.

In [24]:
def calculate_query_document_scores(query, token_id_dict, id_token_dict):
    doc_scores = defaultdict(int)
    # calculate rankings based on JM smoothing
    for token in query.split():  # for each token in the query...
        list_ids = token_id_dict[token]  # get the list of ids with that token
        for doc_id in list_ids:  # for each document in that list...
            jm_score = jm_smoothing(len(id_token_dict[doc_id]), len(token_id_dict[token])/len(id_token_dict))  # calculate the JM smoothing score for the matched token
            doc_scores[doc_id] += jm_score  # add to documents overall score

    return doc_scores

In [25]:
query = "JOOHN SMIITH"
# uncomment if want query substitution
query = substitute_similar_tokens(query, token_id_dict, JaroWinkler.similarity)

# calculate rankings for all documents
doc_scores = calculate_query_document_scores(query, token_id_dict, id_token_dict)

tie_break_scores = set()
# get the top 10 results in the initial JM score rankings
print(f"QUERY Sent: {query}\n")
for doc_id in sorted(doc_scores, key=doc_scores.get, reverse=True)[:10]:
    print(doc_id, doc_scores[doc_id])
    tie_break_scores.add(doc_scores[doc_id])

QUERY Sent: JOHN SMITH

3958 10.075826093753555
6376 8.70780814555837
348 5.9234046817938175
748 5.9234046817938175
1356 5.9234046817938175
1689 5.9234046817938175
2252 5.9234046817938175
2509 5.9234046817938175
2809 5.9234046817938175
3123 5.9234046817938175


### DOCUMENT RANKING NAME SIMILARITY TIEBREAKERS

The second use of string similarity metrics in the process, this serves as an additional ranking to be applied after the JM Smoothing ranking has been calculated and the top results returned. For each of the top 10 scores I note the score associated, and then all IDs of those scores are gathered and their ranking recalculated with a different metric. For my implementation I took the MAX of all the scores created by comparing the query to all the document names, but the AVG could be taken as well (although there are pros and cons to both). After a MAX similarity score has been calculated for each ID, the results are re-ranked and the final ranking is provided.  
<br>
#### KEY ASSUMPTIONS:
- comparing the entire query to each document name with Jaro-Winkler is a good similarity measure
- query names and document names are in the same order ("JOHN SMITH" <> "SMITH JOHN")
- MAX similarity score over AVG similarity score
- arbitrary ordering for tied similarity score
- top 10 JM SMoothing results is a large enough window! For these specific examples, the assumptions called out in earlier portions show their effects... specifically, that a document is NOT more likely to have names that are similar to each other. As a result, this implementation is less likely to match to IDs with 2+ names for queries with only 2 tokens, since the resulting discount from document length in the JM Smoothing algorithm leads to them ranking lower. That said, queries with 3+ tokens will have a higher probability to show documents with 3+ tokens as well

In [26]:
def calculate_query_document_tiebreaks(query, tie_break_ids, df_corpus, similarity_function):
    doc_sim_scores = {}

    for doc_id in tie_break_ids:  # for each document considered for tiebreaks...
        max_sim_score = 0
        for doc_name in df_corpus[df_corpus['id'] == doc_id]['name']:  # get each associated name
            max_sim_score = max(max_sim_score, similarity_function(query, doc_name))  # get max similarity score between name and query
        doc_sim_scores[doc_id] = max_sim_score

    return doc_sim_scores

In [27]:
score_doc_ids = defaultdict(list)
tie_break_ids = []

# create reverse dictionary of each id associated with certain final JM score (for tie breaking)
for doc_id in doc_scores.keys():
    score_doc_ids[doc_scores[doc_id]].append(doc_id)

# get list of ids associated with tiebreak scores
for score in tie_break_scores:
    tie_break_ids += score_doc_ids[score]

# calculate tie-break rankings for all documents with score from JM top 10
doc_sim_scores = calculate_query_document_tiebreaks(query, tie_break_ids, df_corpus, JaroWinkler.similarity)

In [28]:
# get top 10 results in tiebreak rankings
for doc_id in sorted(doc_sim_scores, key=doc_sim_scores.get, reverse=True)[:10]:
    print(f"df: {df_corpus[df_corpus.id == doc_id]}\nmax_sim_score:{doc_sim_scores[doc_id]}\n")

df:                 name    id             tokens
558       JOHN SMITH  6376      [JOHN, SMITH]
9441  JORDAN SIMMONS  6376  [JORDAN, SIMMONS]
avg_sim_score:1.0

df:             name    id         tokens
8558  JOHN SMITH  3958  [JOHN, SMITH]
avg_sim_score:1.0

df:             name   id         tokens
8577  JOHN HICKS  348  [JOHN, HICKS]
avg_sim_score:0.895

df:             name    id         tokens
8839  JOHN MILLS  4412  [JOHN, MILLS]
avg_sim_score:0.895

df:            name    id        tokens
8258  JOHN REID  7989  [JOHN, REID]
avg_sim_score:0.8533333333333333

df:            name    id        tokens
3931  JOHN TRAN  8029  [JOHN, TRAN]
avg_sim_score:0.8533333333333333

df:             name    id         tokens
2413  JOHN ORTIZ  1356  [JOHN, ORTIZ]
avg_sim_score:0.8514285714285714

df:             name    id         tokens
2886  JOHN DAVIS  2809  [JOHN, DAVIS]
avg_sim_score:0.8514285714285714

df:             name    id         tokens
2723  JOHN RAMOS  4316  [JOHN, RAMOS]
avg_sim_scor